# Yelp_API_CORE

## Import and Load

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [75]:
import json
with open('/Users/ashleyheinrich/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

## API

In [76]:
# set our API call parameters
LOCATION = 'Charlotte, NC'
TERM = 'restaurants'
CATEGORIES = 'italian'

In [77]:
# specifying JSON file
JSON_FILE = "Data/results_in_progress_NC_restaurant_italian.json"
JSON_FILE

'Data/results_in_progress_NC_restaurant_italian.json'

**Defining Function**

In [78]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
                
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

**Calling Function**

In [80]:
# Create a new empty json file (exist the previous if it exists)

create_json_file(JSON_FILE, delete_if_exists=True)

## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
### set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

#### use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                categories = CATEGORIES,
                               offset=n_results)

# How many results total?
total_results = results['total']

## How many did we get the details for?
results_per_page = len(results['businesses'])

### Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NC_restaurant_italian.json already exists. Deleting previous file...
[i] Data/results_in_progress_NC_restaurant_italian.json not found. Saving empty list to new file.
- 0 previous results found.


4

**Stop Loop**

In [82]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    categories = CATEGORIES,
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/4 [00:00<?, ?it/s]

**Create Dataframe**

In [84]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,VJ4fKEeTljlRgCTfYdcs_Q,north-italia-charlotte-charlotte-3,North Italia- Charlotte,https://s3-media1.fl.yelpcdn.com/bphoto/gXXnO6...,False,https://www.yelp.com/biz/north-italia-charlott...,534,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 35.217367419856366, 'longitude': ...",[delivery],$$,"{'address1': '1414 S Tryon St', 'address2': 'S...",+19802798900,(980) 279-8900,2765.647287
1,xtI0IrtdusfRNhr91Gvw8Q,cicchetti-charlotte,Cicchetti,https://s3-media4.fl.yelpcdn.com/bphoto/sFn1qY...,False,https://www.yelp.com/biz/cicchetti-charlotte?a...,56,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.5,"{'latitude': 35.2264258325475, 'longitude': -8...",[delivery],$$,"{'address1': '100 N Tryon St', 'address2': '',...",+19802990039,(980) 299-0039,2680.884708
2,A_fpNRoUZRU41n8_9LJJMA,little-mamas-charlotte-2,Little Mama's,https://s3-media2.fl.yelpcdn.com/bphoto/jpMRrv...,False,https://www.yelp.com/biz/little-mamas-charlott...,377,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 35.150636, 'longitude': -80.828631}","[pickup, delivery]",$$,"{'address1': '4521 Sharon Rd', 'address2': '',...",+19802090323,(980) 209-0323,6021.377797
3,USleI8g1UVN9-9jm-b4uIg,the-fig-tree-restaurant-charlotte,The Fig Tree Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/Ie_510...,False,https://www.yelp.com/biz/the-fig-tree-restaura...,579,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 35.21657923041921, 'longitude': -...",[delivery],$$$,"{'address1': '1601 E 7th St', 'address2': None...",+17043323322,(704) 332-3322,1430.586445
4,sUYTbo8MmmLytFTKgME6PQ,ever-andalo-charlotte,Ever Andalo,https://s3-media1.fl.yelpcdn.com/bphoto/JLr07W...,False,https://www.yelp.com/biz/ever-andalo-charlotte...,104,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 35.2467213623962, 'longitude': -8...",[],$$$,"{'address1': '3116 N Davidson St', 'address2':...",+17049106543,(704) 910-6543,5109.946438


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
59,-8Oaw94x2pYJCVP8ZtuTTw,carrabbas-italian-grill-matthews-6,Carrabba's Italian Grill,https://s3-media3.fl.yelpcdn.com/bphoto/7KasRi...,False,https://www.yelp.com/biz/carrabbas-italian-gri...,211,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.0,"{'latitude': 35.127594118220664, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '10408 E Independence Blvd', 'add...",+17048440464,(704) 844-0464,13950.778357
60,8QRf5qGDEuDulIIRzZqL2w,bellacinos-pizza-and-grinders-charlotte-2,Bellacino's Pizza and Grinders,,False,https://www.yelp.com/biz/bellacinos-pizza-and-...,0,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",0.0,"{'latitude': 35.2251625061035, 'longitude': -8...",[delivery],NaN,"{'address1': '333 E Trade St', 'address2': Non...",+17043727122,(704) 372-7122,2457.254349
61,oYDE-FP4oAjaM6KQk3FM2Q,my-sausage-buddy-charlotte,My Sausage Buddy,https://s3-media2.fl.yelpcdn.com/bphoto/GITEOd...,False,https://www.yelp.com/biz/my-sausage-buddy-char...,12,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.0,"{'latitude': 35.12061, 'longitude': -80.71788}",[],NaN,"{'address1': None, 'address2': None, 'address3...",+15165120574,(516) 512-0574,15033.097307
62,281eGXO-yxDMBIggVA7eDQ,zinos-charlotte,Zino's,,False,https://www.yelp.com/biz/zinos-charlotte?adjus...,0,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",0.0,"{'latitude': 35.1622956991196, 'longitude': -8...",[],NaN,"{'address1': '7308 E Independence Blvd', 'addr...",+17045681999,(704) 568-1999,4589.109133
63,Pd-CDiYN7g1OJ-VJfl_NYg,sapori-d-ithalia-charlotte,Sapori d'Ithalia,https://s3-media1.fl.yelpcdn.com/bphoto/rnx8Nv...,False,https://www.yelp.com/biz/sapori-d-ithalia-char...,0,"[{'alias': 'personalchefs', 'title': 'Personal...",0.0,"{'latitude': 35.17145414589084, 'longitude': -...",[],NaN,"{'address1': '', 'address2': None, 'address3':...",+19803370581,(980) 337-0581,8116.725354


# New heading